The following cell imports the packages that you will need.

In [2]:
import glob
import os.path
import collections
import numpy
import netCDF4
from gewittergefahr.gg_utils import error_checking

The following cell defines the constants that you will need.

In [ ]:
NUM_BATCHES_PER_DIRECTORY = 1000
BATCH_NUMBER_REGEX = '[0-9][0-9][0-9][0-9][0-9][0-9][0-9]'

TEMPERATURE_NAME = 'temperature_kelvins'
HEIGHT_NAME = 'height_m_asl'
SPECIFIC_HUMIDITY_NAME = 'specific_humidity_kg_kg01'
WET_BULB_THETA_NAME = 'wet_bulb_potential_temperature_kelvins'
U_WIND_GRID_RELATIVE_NAME = 'u_wind_grid_relative_m_s01'
V_WIND_GRID_RELATIVE_NAME = 'v_wind_grid_relative_m_s01'

VALID_PREDICTOR_NAMES = [
    TEMPERATURE_NAME, HEIGHT_NAME, SPECIFIC_HUMIDITY_NAME, WET_BULB_THETA_NAME,
    U_WIND_GRID_RELATIVE_NAME, V_WIND_GRID_RELATIVE_NAME
]

PREDICTOR_MATRIX_KEY = 'predictor_matrix'
TARGET_MATRIX_KEY = 'target_matrix'
TARGET_TIMES_KEY = 'target_times_unix_sec'
ROW_INDICES_KEY = 'row_indices'
COLUMN_INDICES_KEY = 'column_indices'
PREDICTOR_NAMES_KEY = 'narr_predictor_names'
NARR_MASK_KEY = 'narr_mask_matrix'
PRESSURE_LEVEL_KEY = 'pressure_level_mb'
DILATION_DISTANCE_KEY = 'dilation_distance_metres'

The following cell defines private methods that you will need.  Since this is a notebook and not a proper Python package, there is really no distinction between public and private methods.  However, I have used the syntax for private methods (underscore at the beginning of the method name), to emphasize that these are low-level helper methods and you shouldn't worry about them.

In [3]:
def _check_predictor_name(field_name):
    """Ensures that name of model field is recognized.

    :param field_name: Field name in GewitterGefahr format (not the original
        NetCDF format).
    :raises: ValueError: if field name is unrecognized.
    """

    error_checking.assert_is_string(field_name)

    if field_name not in VALID_PREDICTOR_NAMES:
        error_string = (
            '\n\n' + str(VALID_PREDICTOR_NAMES) +
            '\n\nValid field names (listed above) do not include "' +
            field_name + '".')
        raise ValueError(error_string)


def _floor_to_nearest(input_value, rounding_base):
    """Rounds numbers *down* to nearest x, where x is a positive real number.

    :param input_value: Either numpy array of real numbers or scalar real
        number.
    :param rounding_base: Numbers will be rounded *down* to this base.
    :return: output_value: Same as input_value, except rounded.
    """

    if isinstance(input_value, collections.Iterable):
        error_checking.assert_is_real_numpy_array(input_value)
    else:
        error_checking.assert_is_real_number(input_value)

    error_checking.assert_is_greater(rounding_base, 0)
    return rounding_base * numpy.floor(input_value / rounding_base)


def _file_name_to_batch_number(downsized_3d_file_name):
    """Parses file name for batch number.

    :param downsized_3d_file_name: See doc for `find_downsized_3d_example_file`.
    :return: batch_number: Integer.
    :raises: ValueError: if batch number cannot be parsed from file name.
    """

    pathless_file_name = os.path.split(downsized_3d_file_name)[-1]
    extensionless_file_name = os.path.splitext(pathless_file_name)[0]
    return int(extensionless_file_name.split('downsized_3d_examples_batch')[-1])

The following method shrinks the dimensions of a training examples.  The original examples (stored in the files) are 65 rows (latitudes) x 65 columns (longitudes).  Shrinking the grids makes them easier to work with.  The grid dimensions must always be odd numbers, which is why the input arguments are num_half_rows and num_half_columns, rather than num_rows and num_columns.  This ensures that there is exactly one center grid cell, which is the grid cell whose label (no front, warm front, or cold front) we are trying to predict.  For example, if you want to shrink the grids to 33 x 33, make num_half_rows=16 and num_half_columns=16.  The grids will be cropped around the center, so the center grid cell will remain the same.  It's just the number of surrounding grid cells that may shrink.

In [4]:
def decrease_example_size(predictor_matrix, num_half_rows, num_half_columns):
    """Decreases the grid size for each example.

    M = original number of rows per example
    N = original number of columns per example
    m = new number of rows per example
    n = new number of columns per example

    :param predictor_matrix: E-by-M-by-N-by-C numpy array of predictor images.
    :param num_half_rows: Determines number of rows returned for each example.
        Examples will be cropped so that the center of the original image is the
        center of the new image.  If `num_half_rows`, examples will not be
        cropped.
    :param num_half_columns: Same but for columns.
    :return: predictor_matrix: E-by-m-by-n-by-C numpy array of predictor images.
    """

    if num_half_rows is not None:
        error_checking.assert_is_integer(num_half_rows)
        error_checking.assert_is_greater(num_half_rows, 0)

        center_row_index = int(
            numpy.floor(float(predictor_matrix.shape[1]) / 2)
        )
        first_row_index = center_row_index - num_half_rows
        last_row_index = center_row_index + num_half_rows
        predictor_matrix = predictor_matrix[
            :, first_row_index:(last_row_index + 1), ...
        ]

    if num_half_columns is not None:
        error_checking.assert_is_integer(num_half_columns)
        error_checking.assert_is_greater(num_half_columns, 0)

        center_column_index = int(
            numpy.floor(float(predictor_matrix.shape[2]) / 2)
        )
        first_column_index = center_column_index - num_half_columns
        last_column_index = center_column_index + num_half_columns
        predictor_matrix = predictor_matrix[
            :, :, first_column_index:(last_column_index + 1), ...
        ]

    return predictor_matrix

The following method locates a file with training examples.  On average each file contains 512 training examples: 256 NF examples (with no front at the center grid cell), 128 WF examples (warm front at the center grid cell), and 128 CF examples (cold front at the center grid cell).  The original class distribution is much more skewed (98.95% of examples are NF), which makes the deep-learning model nearly insensitive to the minority classes (WF and CF), which leads to the predicted probabilities of WF and CF always being very low.  Balancing the training data fixes the problem.  Unfortunately it causes the DL models to overpredict the WF and CF classes, but this can be mitigated by post-processing.

In [5]:
def find_downsized_3d_example_file(
        top_directory_name, batch_number, raise_error_if_missing=True):
    """Finds file with downsized 3-D examples.

    :param top_directory_name: Name of top-level directory for files with
        downsized 3-D examples.
    :param batch_number: Batch number (integer).
    :param raise_error_if_missing: Boolean flag.  If file is missing and
        `raise_error_if_missing = True`, this method will error out.
    :return: downsized_3d_file_name: Path to file with downsized 3-D examples.
        If file is missing and `raise_error_if_missing = False`, this is the
        *expected* path.
    :raises: ValueError: if file is missing and `raise_error_if_missing = True`.
    """

    error_checking.assert_is_string(top_directory_name)
    error_checking.assert_is_boolean(raise_error_if_missing)

    error_checking.assert_is_integer(batch_number)
    error_checking.assert_is_geq(batch_number, 0)

    first_batch_number = int(_floor_to_nearest(
        batch_number, NUM_BATCHES_PER_DIRECTORY))
    last_batch_number = first_batch_number + NUM_BATCHES_PER_DIRECTORY - 1

    downsized_3d_file_name = (
        '{0:s}/batches{1:07d}-{2:07d}/downsized_3d_examples_batch{3:07d}.nc'
    ).format(top_directory_name, first_batch_number, last_batch_number,
             batch_number)

    if raise_error_if_missing and not os.path.isfile(downsized_3d_file_name):
        error_string = 'Cannot find file.  Expected at: "{0:s}"'.format(
            downsized_3d_file_name)
        raise ValueError(error_string)

    return downsized_3d_file_name

This method locates many files with training examples.

In [6]:
def find_downsized_3d_example_files(
        top_directory_name, first_batch_number, last_batch_number):
    """Finds many files with downsized 3-D examples.

    :param top_directory_name: See doc for `find_downsized_3d_example_file`.
    :param first_batch_number: First batch number.
    :param last_batch_number: Last batch number.
    :return: downsized_3d_file_names: 1-D list of file paths.
    :raises: ValueError: if no files are found.
    """

    error_checking.assert_is_string(top_directory_name)

    error_checking.assert_is_integer(first_batch_number)
    error_checking.assert_is_integer(last_batch_number)
    error_checking.assert_is_geq(first_batch_number, 0)
    error_checking.assert_is_geq(last_batch_number, first_batch_number)

    downsized_3d_file_pattern = (
        '{0:s}/batches{1:s}-{1:s}/downsized_3d_examples_batch{1:s}.nc'
    ).format(top_directory_name, BATCH_NUMBER_REGEX)

    downsized_3d_file_names = glob.glob(downsized_3d_file_pattern)
    if len(downsized_3d_file_names) == 0:
        error_string = 'Cannot find any files with the pattern: "{0:s}"'.format(
            downsized_3d_file_pattern)
        raise ValueError(error_string)

    batch_numbers = numpy.array(
        [_file_name_to_batch_number(f) for f in downsized_3d_file_names],
        dtype=int)
    good_indices = numpy.where(numpy.logical_and(
        batch_numbers >= first_batch_number,
        batch_numbers <= last_batch_number
    ))[0]

    if len(good_indices) == 0:
        error_string = (
            'Cannot find any files with batch number in [{0:d}, {1:d}].'
        ).format(first_batch_number, last_batch_number)
        raise ValueError(error_string)

    return [downsized_3d_file_names[i] for i in good_indices]

This method reads a file with training examples.

In [7]:
def read_downsized_3d_examples(
        netcdf_file_name, metadata_only=False, predictor_names_to_keep=None,
        num_half_rows_to_keep=None, num_half_columns_to_keep=None):
    """Reads downsized 3-D examples from NetCDF file.

    :param netcdf_file_name: Path to input file.
    :param metadata_only: Boolean flag.  If True, will return only metadata
        (everything except predictor and target matrices).
    :param predictor_names_to_keep: 1-D list with names of predictor variables
        to keep (each name must be accepted by `_check_predictor_name`).  If
        `predictor_names_to_keep is None`, all predictors in the file will be
        returned.
    :param num_half_rows_to_keep: [used iff `metadata_only == False`]
        Determines number of rows to keep for each example.  Examples will be
        cropped so that the center of the original image is the center of the
        new image.  If `num_half_rows_to_keep is None`, examples will not be
        cropped.
    :param num_half_columns_to_keep: [used iff `metadata_only == False`]
        Same but for columns.
    :return: example_dict: Dictionary with the following keys.
    example_dict['predictor_matrix']: See doc for
        `prep_downsized_3d_examples_to_write`.
    example_dict['target_matrix']: Same.
    example_dict['target_times_unix_sec']: Same.
    example_dict['row_indices']: Same.
    example_dict['column_indices']: Same.
    example_dict['predictor_names_to_keep']: See doc for
        `write_downsized_3d_examples`.
    example_dict['pressure_level_mb']: Same.
    example_dict['dilation_distance_metres']: Same.
    example_dict['narr_mask_matrix']: Same.
    """

    error_checking.assert_is_boolean(metadata_only)
    if predictor_names_to_keep is not None:
        error_checking.assert_is_numpy_array(
            numpy.array(predictor_names_to_keep), num_dimensions=1)
        for this_name in predictor_names_to_keep:
            _check_predictor_name(this_name)

    netcdf_dataset = netCDF4.Dataset(netcdf_file_name)

    narr_predictor_names = netCDF4.chartostring(
        netcdf_dataset.variables[PREDICTOR_NAMES_KEY][:])
    narr_predictor_names = [str(s) for s in narr_predictor_names]
    if predictor_names_to_keep is None:
        predictor_names_to_keep = narr_predictor_names + []

    target_times_unix_sec = numpy.array(
        netcdf_dataset.variables[TARGET_TIMES_KEY][:], dtype=int)
    row_indices = numpy.array(
        netcdf_dataset.variables[ROW_INDICES_KEY][:], dtype=int)
    column_indices = numpy.array(
        netcdf_dataset.variables[COLUMN_INDICES_KEY][:], dtype=int)

    if not metadata_only:
        predictor_matrix = numpy.array(
            netcdf_dataset.variables[PREDICTOR_MATRIX_KEY][:])
        target_matrix = numpy.array(
            netcdf_dataset.variables[TARGET_MATRIX_KEY][:])

        these_indices = numpy.array(
            [narr_predictor_names.index(p) for p in predictor_names_to_keep],
            dtype=int)
        predictor_matrix = predictor_matrix[..., these_indices]
        predictor_matrix = decrease_example_size(
            predictor_matrix=predictor_matrix,
            num_half_rows=num_half_rows_to_keep,
            num_half_columns=num_half_columns_to_keep)

    example_dict = {
        TARGET_TIMES_KEY: target_times_unix_sec,
        ROW_INDICES_KEY: row_indices,
        COLUMN_INDICES_KEY: column_indices,
        PREDICTOR_NAMES_KEY: predictor_names_to_keep,
        PRESSURE_LEVEL_KEY: int(getattr(netcdf_dataset, PRESSURE_LEVEL_KEY)),
        DILATION_DISTANCE_KEY: getattr(netcdf_dataset, DILATION_DISTANCE_KEY),
        NARR_MASK_KEY:
            numpy.array(netcdf_dataset.variables[NARR_MASK_KEY][:], dtype=int)
    }

    if metadata_only:
        netcdf_dataset.close()
        return example_dict

    example_dict.update({
        PREDICTOR_MATRIX_KEY: predictor_matrix.astype('float32'),
        TARGET_MATRIX_KEY: target_matrix.astype('float64')
    })

    netcdf_dataset.close()
    return example_dict